In [7]:
import os, csv, json, requests
import glob
import pycountry
import pandas as pd
import numpy as np
from datetime import date
import pickle

In [8]:
coord = pd.read_csv("Coordinate_unique.csv")

In [9]:
def generate_pv_prod_data(coord):
    url_api = "https://re.jrc.ec.europa.eu/api/v5_2/PVcalc?"
    mktrgn_list = ["RUS","CHN","RSAF","INDO","JAP","RSAM","WAF","UKR","INDIA","ME","WEU","NAF","KOR","EAF","STAN","CEU","RCAM","CAN","RSAS","SAF","BRA","TUR","OCE","USA","SEAS"]
    tech_list = ['crystSi', 'CIS', 'CdTe', 'Unknown']
    mounting_list = ['free', 'building']
    angle_list = ['90','0']
    optincl = '1'
    poss_list = []
    test_list = []
    url_list = []
    yrlygen_list = []
    result_list = []
    
    p0 = "&peakpower=1"
    p1 = "&pvtechchoice="
    p2 = "&mountingplace="
    p3 = "&angle="
    p4 = "&optimalinclination="
    mp = "&loss=14&outputformat=json"
    
    # Boucle pour générer chaque suite et les stocker dans la troisième liste
    for e1 in tech_list:
        for e2 in mounting_list:
            if e2 == "free":
                suite_url = p0+p1+e1+p2+e2+p4+optincl+mp
                poss_list.append((suite_url, e1+" "+e2))
            else:
                for e3 in angle_list:
                    suite_url = p0+p1+e1+p2+e2+p3+e3+mp
                    poss_list.append((suite_url,e1+" "+e2+" "+e3))
    #print(poss_list)
    for region in mktrgn_list:
        coordonnees_region = coord.loc[coord['Short name'] == region, ['Latitude', 'Longitude']]
        for x in poss_list:
            for index, row in coordonnees_region.iterrows():
                lat = "lat="+"{:.2f}".format(row['Latitude'])+"&"
                lon = "lon="+"{:.2f}".format(row['Longitude'])
                url_request = url_api + lat + lon + x[0]
                #url_list.append(url_request) 
                response_test = requests.get(url_request)
                jsondata_test = response_test.json()
                if response_test.status_code == 200:
                    #print("Success")
                    jsondata_test = response_test.json()
                    yrlygen_list.append(jsondata_test["outputs"]["totals"]["fixed"]["E_y"])
            mean_E_y = np.mean(yrlygen_list)
            yrlygen_list = []
            result_list.append((region, x[1], mean_E_y))
            #print(region,x[1],mean_E_y)
    return result_list
                    

In [10]:
with open(r"pv_detailsSSP2-RCP19-2050.pickle", "rb") as input_file:
        pv_pickle = pickle.load(input_file)

In [12]:
def add_custom_production_data_to_df(pv_pickle, result_list):
    for index, row in pv_pickle.iterrows():
        for r in result_list:
            if row['Market group location'] == r[0]:
                tech_temp = row['Technology']
                mount_temp = row['Installation']
                pvtechchoice = (" crystSi" if "Si" in tech_temp
                        else " CIS" if tech_temp == " CIS"
                        else " CdTe" if tech_temp == " CdTe"
                        else " Unknown")
                mountingplace = (" building" if "roof" in mount_temp
                        else " building" if "facade" in mount_temp
                        else " free")
                angle = (" 0" if "roof" in mount_temp
                        else " 90" if "facade" in mount_temp
                        else "")
                ds_temp = str(row['Market group location'])+pvtechchoice+mountingplace+angle
                if ds_temp == str(r[0])+" "+str(r[1]):
                    pv_pickle.at[index, 'Annual production'] = r[2]
    return pv_pickle

In [14]:
add_custom_production_data_to_df(pv_pickle, generate_pv_prod_data(coord))
pv_pickle

C:\Users\edgar\virtualenvs\MPTEX-proj\Lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\edgar\virtualenvs\MPTEX-proj\Lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


,Market Dataset,ecoinvent location,Market group location,Peak power (kWp),Installation,Technology,Placement,Build,Placement2,Annual production
0,"market for electricity, low voltage, renewable...",CH,NaN,3,slanted-roof,single-Si,panel,mounted,on roof,1664.09
1,"market for electricity, low voltage, renewable...",CH,NaN,3,slanted-roof,multi-Si,panel,mounted,on roof,1795.22
0,"market for electricity, low voltage",BJ,WAF,3,slanted-roof,CIS,laminated,integrated,on roof,1664.09
1,"market for electricity, low voltage",BJ,WAF,3,slanted-roof,CdTe,panel,mounted,on roof,1795.22
2,"market for electricity, low voltage",BJ,WAF,3,slanted-roof,micro-Si,laminated,integrated,on roof,1640.68
...,...,...,...,...,...,...,...,...,...,...
14116,"market for electricity, high voltage",CN-CSG,CHN,570,open ground,single-Si,on open ground,None,None,1368.54
14117,"market for electricity, high voltage",CN-CSG,CHN,93,slanted-roof,multi-Si,laminated,integrated,on roof,1145.64
14118,"market for electricity, high voltage",CN-CSG,CHN,93,slanted-roof,multi-Si,panel,mounted,on roof,1145.64
14119,"market for electricity, high voltage",CN-CSG,CHN,93,slanted-roof,single-Si,laminated,integrated,on roof,1145.64
